In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device='cuda:1'
# 加载模型和分词器
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")


In [11]:
def basic_generation(prompt: str) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # 生成回复（设置生成参数）
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,       # 最大生成token数
        temperature=0.7,          # 控制随机性（0-1，越高越随机）
        top_p=0.9,                # 核采样参数
        do_sample=True,           # 启用采样
        pad_token_id=tokenizer.eos_token_id  # 避免警告
    )
    
    # 解码并返回结果
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [12]:
# question='In power systems, we will forecasts the load in the next and dispatch generators, is the impact of overestimation and underestimation of load\
# the same on downstream decision-making?'

# response = basic_generation(question)

In [ ]:
# Refer to the document for workspace information: https://help.aliyun.com/document_detail/2746874.html    
        
from http import HTTPStatus
import dashscope
 
def call_with_stream():
    messages = [
        {'role': 'user', 'content': '介绍一下你自己'}]
    responses = dashscope.Generation.call("qwen-max", #调用的模型接口
                                messages=messages,
                                result_format='message',  # set the result to be "message"  format.
                                stream=True, # 是否开启流式输出，不开启设置False
                                incremental_output=True  # 是否开启流式输出，不开启设置False
                                )
    for response in responses:
        if response.status_code == HTTPStatus.OK:
            print(response.output.choices[0]['message']['content'],end='')
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
 
 
if __name__ == '__main__':
        call_with_stream()

In [13]:
import numpy as np

class CurveDictDB:
    def __init__(self):
        self.data = {}   # key: id, value: 记录字典
    
    def insert(self, record):
        """
        插入一条新的实验记录，id 必须唯一
        """
        record_id = record['id']
        if record_id in self.data:
            raise ValueError(f"id {record_id} 已存在，不能重复插入！")
        self.data[record_id] = record
    
    def get_top_k(self, curve_new, k=3, distance_type='euclidean'):
        """
        查找与 curve_new 最近的 k 条记录
        """
        results = []
        for record_id, record in self.data.items():
            pre_curve = record['load']
            if distance_type == 'euclidean':
                dist = np.linalg.norm(np.array(curve_new) - np.array(pre_curve))
            elif distance_type == 'manhattan':
                dist = np.sum(np.abs(np.array(curve_new) - np.array(pre_curve)))
            else:
                raise ValueError(f"未知的distance_type: {distance_type}")
            results.append({
                'id': record_id,
                'prompt': record['prompt'],
                'distance': dist,
                'record': record
            })
        results.sort(key=lambda x: x['distance'])
        return results[:k]

    def update(self, record_id, new_data):
        """
        更新指定 id 的实验记录
        """
        if record_id in self.data:
            self.data[record_id].update(new_data)
            return True
        return False

    def get_by_id(self, record_id):
        """
        根据 id 获取实验记录
        """
        return self.data.get(record_id)  # 找不到返回 None

    def all(self):
        """
        返回所有记录的列表
        """
        return list(self.data.values())

# ------ 使用举例 -------

db = CurveDictDB()
# 插入三条数据
db.insert({'id':1,'flag': 1, 'prompt': 'a', 'load': [0.1]*24,'fine_tune_load': [0.1]*24, 'cost': 3.2})
db.insert({'id':2,'flag': 0, 'prompt': 'b', 'load': [0.5]*24, 'fine_tune_load': [0.5]*24, 'cost': 5.1})
db.insert({'id':3,'flag': 0, 'prompt': 'c', 'load': [0.2]*24, 'fine_tune_load': [0.2]*24, 'cost': 6.7})

curve_new = [0.15]*24

topk = db.get_top_k(curve_new, k=2)
for i, item in enumerate(topk):
    print(f"第{i+1}条: id={item['id']}, prompt={item['prompt']}, distance={item['distance']:.4f}")

# 更新
db.update(1, {'cost': 9.9})

# 查询
print("id=1的新数据：", db.get_by_id(1))


第1条: id=1, prompt=a, distance=0.2449
第2条: id=3, prompt=c, distance=0.2449
id=1的新数据： {'id': 1, 'flag': 1, 'prompt': 'a', 'load': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 'fine_tune_load': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 'cost': 9.9}


In [18]:
class CurvePromptBuilder:
    def __init__(self):
        pass

    @staticmethod
    def format_curve(curve):
        """
        Formats the curve array as a string.
        """
        return "[" + ", ".join(f"{x:.4f}" for x in curve) + "]"

    def format_record(self, record):
        """
        record: a single record dictionary from CurveDictDB
        """
        text = f"Record ID: {record.get('id', '')}\n"
        text += f"This curve has been fine-tuned by LLM: {record.get('flag', '')}\n"
        text += f"Prompt used at the time: {record.get('prompt', '')}\n"
        text += f"Cost: {record.get('cost', '')}\n"
        text += f"Load curve: {self.format_curve(record['load'])}\n"
        return text

    def build_prompt(self, current_curve, topk_results):
        """
        current_curve: The current prediction curve
        topk_results: The list returned by CurveDictDB.get_top_k()
        """
        prompt = "The current prediction curve is: "
        prompt += self.format_curve(current_curve) + "\n"
        prompt += "The Top-K most similar historical records are:\n"
        for idx, item in enumerate(topk_results):
            prompt += f"---- Record {idx+1} ----\n"
            prompt += self.format_record(item['record'])
            prompt += f"Euclidean distance to the current curve: {item['distance']:.4f}\n"
        prompt += "Based on the historical fine-tuning results, please fine-tune the current load curve to achieve a lower cost.\n"
        return prompt


In [19]:
curve_new = [0.15]*24
topk_results = db.get_top_k(curve_new, k=2)  # 用 get_top_k 得到检索结果

builder = CurvePromptBuilder()
prompt = builder.build_prompt(curve_new, topk_results)
print(prompt)

The current prediction curve is: [0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500]
The Top-K most similar historical records are:
---- Record 1 ----
Record ID: 1
This curve has been fine-tuned by LLM: 1
Prompt used at the time: a
Cost: 9.9
Load curve: [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000]
Euclidean distance to the current curve: 0.2449
---- Record 2 ----
Record ID: 3
This curve has been fine-tuned by LLM: 0
Prompt used at the time: c
Cost: 6.7
Load curve: [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000]
Euclidean distance to the current curve: 0.2449
B